In [1]:
import numpy as np
import scipy.stats as stats
import numba

In [2]:
def cis_overlap(ci1, ci2) -> bool:
    return (ci2.low <= ci1.low <= ci2.high) or (ci1.low <= ci2.low <= ci1.high)

In [3]:
n_samples = 500
n_sets = 50
n_resamples = 1000
n_repetitions = 100

## Determining Distribution Equality Using a CI of the Bootstrapped KS Statistic

We sample from two distributions, $A$ and $B$, and attempt to bootstrap a CI on the KS statistic.

Let $X_{A,i}$ be a vector of N samples from distribution A, for $i \in \{1..N_{sets}\}$. "A" is a stand-in for Python simulation code, "B" for the IDL. The "sets" are a stand-in for simulation runs.

Let $K_{j,k}$ be the KS statistic applied to sample vectors j and k.

### Test 1

This is one method that was suggested in our call. I am not entirely sure it is implemented as described. The expectation is that, since sets of samples are from the same distribution, the test would confirm that the distributions are equivalent.

A and B ~ N(0,1)

We bootstrap two 95% confidence intervals: 

$CI_1$ is constructed by sampling from $K_{j,k}, \forall j \in X_{A,i}, \forall k \in X_{A,i}$

$CI_2$ is constructed by sampling from $K_{j,k}, \forall j \in X_{B,i}, \forall k \in X_{B,i}$

We run this process repeatedly, and find that the CIs overlap in 31% of trials. This does not appear to be successful.

In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_A = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_A], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_A = stats.bootstrap(KS_A_A, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_A, CI_B_B))

np.mean(results)

### Test 2

This is a method that I came up with that seems, to me, to be valid. If it is not, please help me to understand why.

A and B ~ N(0,1)

Let $P_{j,k}$ be the p-values resulting from the KS test $\forall j \in X_{A,i}, \forall k \in X_{A,i}$ 

We construct a 95% confidence interval by sampling from $P_{j,k}$

The resulting CI is (0.27, 0.29), suggesting that that the 

In [4]:
def kstest_pvalue(x, y, axis):
    return stats.kstest(x, y).pvalue

In [11]:
results = []

X_A = np.random.normal(loc=0.0, scale=1.0, size=n_sets*n_samples)
X_B = np.random.normal(loc=0.0, scale=1.0, size=n_sets*n_samples)

CI_P_A_B = stats.bootstrap((X_A, X_B), np.vectorize(kstest_pvalue, signature="(n),(n),()->()"), paired=True, batch=5, n_resamples=n_resamples).confidence_interval

In [12]:
CI_P_A_B

ConfidenceInterval(low=0.8865468169185391, high=0.9562510246586273)

In [19]:
results = []

X_A = np.hstack([np.random.normal(loc=x, scale=1.0, size=n_samples//5) for x in np.random.normal(loc=0.0, scale=0.05, size=n_sets)])
X_B = np.hstack([np.random.normal(loc=x, scale=1.0, size=n_samples//5) for x in np.random.normal(loc=0.0, scale=0.03, size=n_sets)])

CI_P_A_B = stats.bootstrap((X_A, X_B), np.vectorize(kstest_pvalue, signature="(n),(n),()->()"), paired=True, batch=5, n_resamples=n_resamples).confidence_interval
CI_P_A_B

ConfidenceInterval(low=0.006607370669901561, high=0.9469108928980675)

### Test 2: A ~ N(0, 1), B ~ N(1, 1)

We repeat the process from Test 1 and find that the CIs overlap in 35% of trials.


In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=1.0, scale=1.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_A = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_A], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_A = stats.bootstrap(KS_A_A, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_A, CI_B_B))

np.mean(results)

### Test 3: A ~ N(0, 1), B ~ N(1, 2)

We repeat the process from Test 1 and find that the CIs overlap in 25% of trials.

In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=1.0, scale=2.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_A = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_A], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_A = stats.bootstrap(KS_A_A, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_A, CI_B_B))

np.mean(results)

## Cross-joined pairs


### Comparing cross-joined to within set 1

In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_B], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_B = stats.bootstrap(KS_A_B, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_B, CI_B_B))

np.mean(results)

In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=1.0, scale=1.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_B], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_B = stats.bootstrap(KS_A_B, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_B, CI_B_B))

np.mean(results)

In [ ]:
results = []

for x in range(n_repetitions):
    X_A = [np.random.normal(loc=0.0, scale=1.0, size=n_samples) for x in range(n_sets)]
    X_B = [np.random.normal(loc=1.0, scale=1.0, size=n_samples) for x in range(n_sets)]

    # Calculate KS statistics for all X_A and X_B, reshape for use in bootstrapping
    KS_A_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_A for y in X_B], axis=0)
    KS_B_B = np.expand_dims([stats.kstest(x, y).statistic for x in X_B for y in X_B], axis=0)

    # Bootstrap two CIs
    CI_A_B = stats.bootstrap(KS_A_B, np.mean, n_resamples=n_resamples).confidence_interval
    CI_B_B = stats.bootstrap(KS_B_B, np.mean, n_resamples=n_resamples).confidence_interval

    # Test for overlap of the CIs
    results.append(cis_overlap(CI_A_B, CI_B_B))
    if x % 5 == 0:
        print(np.mean(results))

np.mean(results)

In [ ]:
n_samples = 500
n_sets = 50

set1 = [np.random.normal(scale=1.0, size=n_samples) for x in range(n_sets)]
set2 = [np.random.normal(scale=1.0, size=n_samples) for x in range(n_sets)]

all_pairs_ks_set1 = [stats.kstest(x, y).statistic for x in set1 for y in set2]
all_pairs_ks_set2 = [stats.kstest(x, y).statistic for x in set1 for y in set1]

ci_set1 = stats.bootstrap(np.array(all_pairs_ks_set1).reshape((1, len(all_pairs_ks_set1))), np.mean, n_resamples=10000)
ci_set2 = stats.bootstrap(np.array(all_pairs_ks_set2).reshape((1, len(all_pairs_ks_set2))), np.mean, n_resamples=10000)

cis_overlap(ci_set1.confidence_interval, ci_set2.confidence_interval)

### Comparing unequal means

In [ ]:
n_samples = 500
n_sets = 50

set1 = [np.random.normal(scale=1.0, size=n_samples) + 1 for x in range(n_sets)]
set2 = [np.random.normal(scale=1.0, size=n_samples) for x in range(n_sets)]

all_pairs_ks_set1 = [stats.kstest(x, y).statistic for x in set1 for y in set2]
all_pairs_ks_set2 = [stats.kstest(x, y).statistic for x in set1 for y in set1]

ci_set1 = stats.bootstrap(np.array(all_pairs_ks_set1).reshape((1, len(all_pairs_ks_set1))), np.mean, n_resamples=10000)
ci_set2 = stats.bootstrap(np.array(all_pairs_ks_set2).reshape((1, len(all_pairs_ks_set2))), np.mean, n_resamples=10000)

cis_overlap(ci_set1.confidence_interval, ci_set2.confidence_interval)

### Comparing unequal variances

In [ ]:
n_samples = 500
n_sets = 50

set1 = [np.random.normal(scale=1.5, size=n_samples) for x in range(n_sets)]
set2 = [np.random.normal(scale=1.0, size=n_samples) for x in range(n_sets)]

all_pairs_ks_set1 = [stats.kstest(x, y).statistic for x in set1 for y in set2]
all_pairs_ks_set2 = [stats.kstest(x, y).statistic for x in set1 for y in set1]

ci_set1 = stats.bootstrap(np.array(all_pairs_ks_set1).reshape((1, len(all_pairs_ks_set1))), np.mean, n_resamples=10000)
ci_set2 = stats.bootstrap(np.array(all_pairs_ks_set2).reshape((1, len(all_pairs_ks_set2))), np.mean, n_resamples=10000)

cis_overlap(ci_set1.confidence_interval, ci_set2.confidence_interval)

In [ ]:
filtered = parsed_df[((parsed_df.fully_successful == 1) | (parsed_df.partially_successful == 1))]
parsed_df["document_text"] = filtered.pagewise_text.str.join("\n")